## Data Viz Notebook - in progress...

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import subplots
from plotly.graph_objs import *
from sklearn import preprocessing

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## Most of this is code from cleaning

I needed to use "income_qol", so I just copied some of the work over

In [2]:
#i stored the csv files in a file named data and this line makes a file path for the .read_csv method
income_csv = os.path.join('data', 'Minimum Wage Data.csv')
qol_csv = os.path.join('data', 'qol_index.csv')
qol_factors = os.path.join('data', 'QOL.xlsx')

#read in the csv files
income_df = pd.read_csv(income_csv)
qol_df = pd.read_csv(qol_csv)
factors = pd.read_excel(qol_factors)

#cleaning the income table
#drop the "Table_Data" and "Footnote" columns
income_df = income_df.drop(columns=['Table_Data', 'Footnote', 'CPI.Average'])

#take only the years after 2011
income_17 = income_df.loc[income_df.Year >= 2011]
income_17['Year'] = income_17['Year'].astype(str)

#find the federal minimums for the year and then remove federal info from table
fed_min = income_17[income_17.State == 'Federal (FLSA)'].reset_index(drop = True)
income_17 = income_17[income_17.State != 'Federal (FLSA)'].reset_index(drop = True)

#need to replace 0s with the federal minimum wage; these are states without a state minimum so federal applies
zero_states = (income_17.loc[(income_17['High.Value'] == 0) & (income_17['Low.Value'] == 0)])
z_idx = zero_states.index
replacing = ['Low.Value','High.Value','High.2018','Low.2018']

#go through each year
for feds in fed_min.index:
    #go through each state at a particular year
    for idx in zero_states.loc[zero_states.Year == fed_min.iloc[feds]['Year']].index:
        #update the 0s in the row to reflect the federal minimum wage at the time
        income_17.loc[idx, replacing] = fed_min.loc[feds].values[2:]

#find the years that the state changed the minimum wage
def find_wage_change(df):
    '''
    Finds the years when a state increased its minimum wage.
    '''
    years = df['High.Value'].diff()[df['High.Value'].diff() != 0].index.values
    if len(years) > 1:
        return df.loc[years]['Year'].values[1:]
                                    
changing_years = income_17.groupby('State').apply(find_wage_change)
changed_df = pd.DataFrame(changing_years, columns = ['Years Changed']).dropna()
changed_df.head()

years_lst = []
years_list = [2012, 2013,2014, 2015, 2016, 2017]
# for each year, find which states changed their minimum wage
for year in years_list:
    for state, years_changed in changed_df.iterrows():
        year = str(year)
        if year in years_changed.values[0]:
            tmp_dict = {}
            tmp_dict.update({'Year':year, 'States': state})
            years_lst.append(tmp_dict)
            
years_changed_df = pd.DataFrame(years_lst).groupby("Year")['States'].apply(list)         
years_changed_df = years_changed_df.to_frame()
years_changed_df

#cleaning of the QOL table
#some general cleaning of the columns for easier handling
qol_df = qol_df.drop(columns = ['FIPS', 'EASI Quality of Life Index, 2018', 'EASI Quality of Life Index, 2019'], axis = 1)
qol_df.columns = qol_df.columns.str.replace('EASI Quality of Life Index, ', '')
qol_17 = qol_df.copy()

#transposing the qol table into a format that's more aligned with the income table
transpose_qol = qol_17.melt('Name')
transpose_qol.columns = ['State', 'Year' , 'QOL Index']
transpose_qol['Year'] = transpose_qol['Year'].astype(int)
transpose_qol['QOL Index'] = transpose_qol['QOL Index'].astype(int)

#find the states that weren't included in both and remove from income_17
no_qol = np.setdiff1d(np.union1d(income_17['State'], transpose_qol['State']), np.intersect1d(income_17['State'], transpose_qol['State']))
income_17 = income_17[~income_17['State'].isin(no_qol)]
income_17.Year = income_17.Year.astype(int)
transpose_qol.Year = transpose_qol.Year.astype(int)
income_qol = income_17.merge(transpose_qol, on = ['State', 'Year'])

income_qol

/Users/johngonzalez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Year,State,High.Value,Low.Value,High.2018,Low.2018,QOL Index
0,2011,Alabama,7.25,7.25,8.07,8.07,109
1,2011,Alaska,7.75,7.75,8.63,8.63,40
2,2011,Arizona,7.35,7.35,8.18,8.18,132
3,2011,Arkansas,6.25,6.25,6.96,6.96,103
4,2011,California,8.00,8.00,8.91,8.91,155
...,...,...,...,...,...,...,...
352,2017,Virginia,7.25,7.25,7.41,7.41,121
353,2017,Washington,11.00,11.00,11.24,11.24,88
354,2017,West Virginia,8.75,8.75,8.94,8.94,54
355,2017,Wisconsin,7.25,7.25,7.41,7.41,58


In [3]:
state_code = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    #'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    #'Palau': 'PW',
    'Pennsylvania': 'PA',
    #'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    #'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

st_code = []

for state in income_qol['State']:
    st_code.append(state_code[state])

#add column with corresponding state abbreviations
codes = np.asarray(st_code)
income_qol['Code'] = codes
income_qol.drop(['High.Value','Low.Value','High.2018','Low.2018','QOL Index'],
                axis=1, inplace=True)
# income_qol.drop(income_qol[income_qol['Year']>2011])

In [4]:
income_qol = income_qol[income_qol['Year']>2011]

In [5]:
income_qol

,Year,State,Code
51,2012,Alabama,AL
52,2012,Alaska,AK
53,2012,Arizona,AZ
54,2012,Arkansas,AR
55,2012,California,CA
...,...,...,...
352,2017,Virginia,VA
353,2017,Washington,WA
354,2017,West Virginia,WV
355,2017,Wisconsin,WI


# Code for map begins here

In [6]:
# for each year, find which states changed their minimum wage

years_lst = []
years_list = [2012, 2013,2014, 2015, 2016, 2017]

for year in years_list:
    for state, years_changed in changed_df.drop('Guam').iterrows():
        state = state_code[state]
        year = str(year)
        if year in years_changed.values[0]:
            tmp_dict = {} 
            tmp_dict.update({'Year':year, 'States': state})
            years_lst.append(tmp_dict)
            
years_changed_df = pd.DataFrame(years_lst, columns=['Year','States']).groupby("Year")['States'].apply(list)         
years_changed_df = years_changed_df.to_frame()
years_changed_df = years_changed_df.reset_index()
years_changed_df

,Year,States
0,2012,"[AZ, CO, FL, MT, OH, OR, VT, WA]"
1,2013,"[AZ, CO, FL, MO, MT, OH, OR, RI, VT, WA]"
2,2014,"[AZ, CA, CO, CT, DE, DC, FL, MI, MN, MO, MT, N..."
3,2015,"[AK, AZ, AR, CO, CT, DE, DC, FL, HI, MD, MA, M..."
4,2016,"[AK, AR, CA, CO, CT, DC, HI, MD, MA, MI, MN, N..."
5,2017,"[AK, AZ, AR, CO, CT, FL, HI, ME, MA, MI, MO, M..."


In [8]:
changed = []

for year in years_changed_df['Year']:
    full_state_list = list(income_qol[income_qol['Year']==int(year)]['Code'])
    changed_state_list = list(years_changed_df[years_changed_df['Year']==year]['States'])
    
    for state in full_state_list:
        if state in changed_state_list[0]:
            changed.append(1)
        else:
            changed.append(0)
    
income_qol.loc[:,'Raised'] = changed

/Users/johngonzalez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/johngonzalez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
orange = [[0.0, '#feedde'],[1.0, '#fd8d3c']]

In [10]:
def map_data(color, year):
    """
    Creates dictionary with information needed to create a choropleth map
    
    Parameters:
        color (str): Color used for map
        year (int): Year selected for values
        
    Returns:
        raised_fig (dict): Dictionary that contains all the necessesary information to plot map
    """
    data = [dict(
                type='choropleth',
                colorscale = color,
                autocolorscale = False,
                locations = income_qol[income_qol['Year']==year]['Code'],   # column with state codes
                z = income_qol[income_qol['Year']==year]['Raised'].astype(float),   # column to be color coded
                locationmode = 'USA-states',
                showscale = False,
                hoverinfo = 'none'
                )]
    
    layout = dict(
                title = 'States That Raised Their Minimum Wage in '+str(year),
                geo = dict(scope='usa',
                           projection=dict(type='albers usa'))
                )
    raised_fig = dict(data=data, layout=layout)
    
    return raised_fig

In [11]:
figs=[]

for yr in range (2012,2018):
    raised_map=map_data(orange, yr)
    figs.append(raised_map)

In [12]:
# create map objects
raised_2012 = go.Figure(figs[0])
raised_2013 = go.Figure(figs[1])
raised_2014 = go.Figure(figs[2])
raised_2015 = go.Figure(figs[3])
raised_2016 = go.Figure(figs[4])
raised_2017 = go.Figure(figs[5])

In [13]:
# create empty subplot
raised_subplot0 = subplots.make_subplots(
                            rows=1,cols=2,
                            specs=[[{"type":"choropleth"},{"type":"choropleth"}]],
                            subplot_titles=('2012','2013')
                                        )

raised_subplot1 = subplots.make_subplots(
                            rows=1,cols=2,
                            specs=[[{"type":"choropleth"},{"type":"choropleth"}]],
                            subplot_titles=('2014','2015')
                                        )

raised_subplot2 = subplots.make_subplots(
                            rows=1,cols=2,
                            specs=[[{"type":"choropleth"},{"type":"choropleth"}]],
                            subplot_titles=('2016','2017',)
                                        )

In [14]:
# add figures to subplot
raised_subplot0.append_trace(raised_2012['data'][0], 1, 1)
raised_subplot0.append_trace(raised_2013['data'][0], 1, 2)

raised_subplot1.append_trace(raised_2014['data'][0], 1, 1)
raised_subplot1.append_trace(raised_2015['data'][0], 1, 2)

raised_subplot2.append_trace(raised_2016['data'][0], 1, 1)
raised_subplot2.append_trace(raised_2017['data'][0], 1, 2)


In [15]:
# set scope to USA
raised_subplot0.update_geos(scope='usa')
raised_subplot1.update_geos(scope='usa')
raised_subplot2.update_geos(scope='usa')

# add title
raised_subplot0.update_layout(title_text='Which States Raised Their Minimum Wage (2012-2013)')
raised_subplot1.update_layout(title_text='Which States Raised Their Minimum Wage (2014-2015)')
raised_subplot2.update_layout(title_text='Which States Raised Their Minimum Wage (2016-2017)')

print()

In [16]:
raised_subplot0.show()
raised_subplot1.show()
raised_subplot2.show()

In [17]:
# import plotly.graph_objects as go
# # Load data frame and tidy it.
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

# st_codes_2012 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][0])
# st_codes_2013 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][1])
# st_codes_2014 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][2])
# st_codes_2015 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][3])
# st_codes_2016 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][4])
# st_codes_2017 = pd.DataFrame(years_changed_df.iloc[ : , 1 ][5])
                    
# fig = go.Figure(data=go.Choropleth(
#     locations= st_codes_2012, # Spatial coordinates
#     zmax=1.0,
#     zmin=0.0,
#     z = ## not sure what to put here...
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Reds',
#     # no color bar 
#     visible = False
# ))

# fig.update_layout(
#     title_text = '2012 States that Changed  Minimum Wage',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()